# Grounding in LTN (cont.)

This tutorial explains how to ground connectives and quantifiers. It expects some familiarity with the first tutorial on grounding non-logical symbols (constants, variables, functions and predicates).

In [1]:
import ltn
import numpy as np
import tensorflow as tf

## Connectives

LTN suppports various logical connectives. They are grounded using fuzzy semantics. We have implemented the most common fuzzy logic operators using tensorflow primitives in `ltn.fuzzy_ops`. We recommend to use the following configuration:
* not: the standard negation  $\lnot u = 1-u$,
* and: the product t-norm $u \land v = uv$,
* or: the product t-conorm (probabilistic sum) $u \lor v = u+v-uv$,
* implication: the Reichenbach implication $u \rightarrow v = 1 - u + uv$,

where $u$ and $v$ denote two truth values in $[0,1]$. For more details on choosing the right operators for your task, read the complementary notebook.

In [2]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())

The wrapper `ltn.Wrapper_Connective` allows to use the operators with LTN formulas. It takes care of combining sub-formulas that have different variables appearing in them (the sub-formulas may have different dimensions that need to be "broadcasted").

In [3]:
x = ltn.Variable('x',np.random.normal(0.,1.,(10,2))) # 10 values in R²
y = ltn.Variable('y',np.random.normal(0.,2.,(5,2))) # 5 values in R²

c1 = ltn.Constant([0.5,0.0], trainable=False)
c2 = ltn.Constant([4.0,2.0], trainable=False)

Eq = ltn.Predicate.Lambda(lambda args: tf.exp(-tf.norm(args[0]-args[1],axis=1))) # predicate measuring similarity

Eq([c1,c2])

ltn.Formula(tensor=0.01775427535176277, free_vars=[])

In [4]:
Not(Eq([c1,c2]))

ltn.Formula(tensor=0.9822457432746887, free_vars=[])

In [5]:
Implies(Eq([c1,c2]), Eq([c2,c1]))

ltn.Formula(tensor=0.9824644327163696, free_vars=[])

In [6]:
# Notice the dimension of the outcome: the result is evaluated for every x. 
And(Eq([x,c1]), Eq([x,c2]))

ltn.Formula(tensor=[0.00901127 0.00025654 0.00037703 0.00124846 0.01122232 0.00096415
 0.01149682 0.00820764 0.0008385  0.00942713], free_vars=['x'])

In [7]:
# Notice the dimensions of the outcome: the result is evaluated for every x and y.
# Notice also that y did not appear in the 1st argument of `Or`; 
# the connective broadcasts the results of its two arguments to match.
Or(Eq([x,c1]), Eq([x,y]))

ltn.Formula(tensor=[[0.34951475 0.3872957  0.35767353 0.806755   0.46382007]
 [0.6583429  0.27672967 0.14298017 0.15572375 0.12005203]
 [0.31957358 0.86959445 0.14900982 0.2441583  0.1774343 ]
 [0.45064244 0.663148   0.2853911  0.39538488 0.29648215]
 [0.37355357 0.40507257 0.38823077 0.7561674  0.4633855 ]
 [0.29771024 0.5812746  0.21300118 0.38740093 0.27458543]
 [0.42646345 0.43158808 0.54606897 0.5007695  0.41557923]
 [0.6621703  0.67272365 0.6594818  0.70286214 0.6302545 ]
 [0.1776562  0.304664   0.14583361 0.36560005 0.32764378]
 [0.7057899  0.7391838  0.694895   0.8211547  0.70261323]], free_vars=['x', 'y'])

## Quantifiers

LTN suppports universal and existential quantification. They are grounded using aggregation operators. We recommend using the following two operators:

- existential quantification ("exists"):  
the generalized mean (`pMean`) $\mathrm{pM}(u_1,\dots,u_n) = \biggl( \frac{1}{n} \sum\limits_{i=1}^n u_i^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$,
- universal quantification ("for all"):  
the generalized mean of "the deviations w.r.t. the truth" (`pMeanError`) $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$,

where $u_1,\dots,u_n$ is a list of truth values in $[0,1]$. 

In [8]:
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=5),semantics="exists")

The wrapper `ltn.Wrapper_Quantifier` allows to use the aggregators with LTN formulas. It takes care of selecting the tensor dimensions to aggregate, given some variables in arguments.

In [9]:
x = ltn.Variable('x',np.random.normal(0.,1.,(10,2))) # 10 values in R²
y = ltn.Variable('y',np.random.normal(0.,2.,(5,2))) # 5 values in R²

Eq = ltn.Predicate.Lambda(lambda args: tf.exp(-tf.norm(args[0]-args[1],axis=1))) # predicate measuring similarity

Eq([x,y])

ltn.Formula(tensor=[[0.13913788 0.04129579 0.4505077  0.11257701 0.00570513]
 [0.07342704 0.04386092 0.8697831  0.08001148 0.00475762]
 [0.06755286 0.05555656 0.7513561  0.0610018  0.00569919]
 [0.44323435 0.04678869 0.14137983 0.07633504 0.01133329]
 [0.13209511 0.18097317 0.15823844 0.02581977 0.02677612]
 [0.08895437 0.01794552 0.33261845 0.22875914 0.00244833]
 [0.15650532 0.07605473 0.34678742 0.06287432 0.01059121]
 [0.06954765 0.1193286  0.34979942 0.0325878  0.011653  ]
 [0.09554081 0.0181297  0.3208171  0.23522575 0.00253532]
 [0.2878225  0.07486501 0.17988682 0.05629261 0.01497131]], free_vars=['x', 'y'])

In [10]:
Forall(x,Eq([x,y]))

ltn.Formula(tensor=[0.14764827 0.06627309 0.34765023 0.09433275 0.0096215 ], free_vars=['y'])

In [11]:
Forall((x,y),Eq([x,y]))

ltn.Formula(tensor=0.12535810470581055, free_vars=[])

In [12]:
Exists((x,y),Eq([x,y]))

ltn.Formula(tensor=0.436784565448761, free_vars=[])

In [13]:
Forall(x, Exists(y, Eq([x,y])))

ltn.Formula(tensor=0.30266571044921875, free_vars=[])

`pMean` can be understood as a smooth-maximum that depends on the hyper-paramer $p$:
- $p \to 1$: the operator tends to `mean`,
- $p \to +\infty$: the operator tends to `max`.

Similarly, `pMeanError` can be understood as a smooth-minimum:
- $p \to 1$: the operator tends to `mean`,
- $p \to +\infty$: the operator tends to `min`.

Therefore, $p$ offers flexibility in writing more or less strict formulas, to account for outliers in the data depending on the application. Note that this can have strong implications for training (see complementary notebook). One can set a default value for $p$ when initializing the operator, or can use different values at each call of the operator.

In [14]:
Forall(x,Eq([x,c1]),p=2)

ltn.Formula(tensor=0.4135896563529968, free_vars=[])

In [15]:
Forall(x,Eq([x,c1]),p=10)

ltn.Formula(tensor=0.33392906188964844, free_vars=[])

In [16]:
Exists(x,Eq([x,c1]),p=2)

ltn.Formula(tensor=0.4701123833656311, free_vars=[])

In [17]:
Exists(x,Eq([x,c1]),p=10)

ltn.Formula(tensor=0.6751344799995422, free_vars=[])

## Diagonal Quantification

Given 2 (or more) variables, there are scenarios where one wants to express statements about specific pairs (or tuples) only, such that the $i$-th tuple contains the $i$-th instances of the variables. We allow this using `ltn.diag`. **Note**: diagonal quantification assumes that the variables have the same number of individuals.

In simplified pseudo-code, the usual quantification would compute:
```python
for x_i in x:
    for y_j in y:
        results.append(P(x_i,y_j))
aggregate(results)
```
In contrast, diagonal quantification would compute:
```python
for x_i, y_i in zip(x,y):
    results.append(P(x_i,y_i))
aggregate(results)
```

We illustrate `ltn.diag` in the following setting: 
- the variable $x$ denotes 100 individuals in $\mathbb{R}^{2\times2}$, 
- the variable $l$ denotes 100 one-hot labels in $\mathbb{N}^3$ (3 possible classes),
- $l$ is grounded according to $x$ such that each pair $(x_i,l_i)$, for $i=0..100$ denotes one correct example from the dataset,
- the classifier $C(x,l)$ gives a confidence value in $[0,1]$ of the sample $x$ corresponding to the label $l$.

In [18]:
# The values are generated at random, for the sake of illustration.
# In a real scenario, they would come from a dataset.
samples = np.random.rand(100,2,2) # 100 R^{2x2} values 
labels = np.random.randint(3, size=100) # 100 labels (class 0/1/2) that correspond to each sample 
onehot_labels = tf.one_hot(labels,depth=3)

x = ltn.Variable("x",samples) 
l = ltn.Variable("l",onehot_labels)

class ModelC(tf.keras.Model):
    def __init__(self):
        super(ModelC, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    def call(self, inputs):
        x, l = inputs[0], inputs[1]
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return tf.math.reduce_sum(x*l,axis=1)

C = ltn.Predicate(ModelC())

If some variables are marked using `ltn.diag`, LTN will only compute their "zipped" results (instead of the usual "broadcasting").

In [19]:
print(C([x,l]).tensor.shape) # Computes the 100x100 combinations
ltn.diag(x,l) # sets the diag behavior for x and l
print(C([x,l]).tensor.shape) # Computes the 100 zipped combinations
ltn.undiag(x,l) # resets the normal behavior
print(C([x,l]).tensor.shape) # Computes the 100x100 combinations

(100, 100)
(100,)
(100, 100)


In practice, `ltn.diag` is designed to be used with quantifiers. 
Every quantifier automatically calls `ltn.undiag` after the aggregation is performed, so that the variables keep their normal behavior outside of the formula.
Therefore, it is recommended to use `ltn.diag` only in quantified formulas as follows.

In [20]:
Forall(ltn.diag(x,l), C([x,l])) # Aggregates only on the 100 "zipped" pairs.
                                # Automatically calls `ltn.undiag` so the behavior of x/l is unchanged outside of this formula.

ltn.Formula(tensor=0.33387333154678345, free_vars=[])

## Guarded Quantifiers

One may wish to quantify over a set of elements whose grounding satisfy some **boolean** condition. Let us assume $x$ is a variable from some domain and $m$ is a mask function that returns boolean values (that is, $0$ or $1$, not continuous truth degrees in $[0,1]$) for each element of the domain.   
In guarded quantification, one has quantifications of the form
$$
(\forall x: m(x)) \phi(x)
$$
which means "every x satisfying $m(x)$ also satisfies $\phi(x)$", or
$$
(\exists x: m(x)) \phi(x)
$$
which means "some x satisfying $m(x)$ also satisfies $\phi(x)$".

The mask $m$ can also depend on other variables in the formula. For instance, the quantification $\exists y (\forall x:m(x,y)) \phi(x,y)$ is also a valid sentence.

Let us consider the following example, that states that there exists an euclidian distance $d$ below which all pairs of points $x$, $y$ should be considered as similar:
$\exists d \ \forall x,y : \mathrm{dist}(x,y) < d \ ( \mathrm{Eq}(x,y))) $

In [21]:
Eq = ltn.Predicate.Lambda(lambda args: tf.exp(-tf.norm(args[0]-args[1],axis=1))) # predicate measuring similarity

points = np.random.rand(50,2) # 50 values in [0,1]^2
x = ltn.Variable("x",points)
y = ltn.Variable("y",points)
d = ltn.Variable("d",[.1,.2,.3,.4,.5,.6,.7,.8,.9])

In [22]:
is_greater_than = ltn.Predicate.Lambda(lambda inputs: inputs[0] > inputs[1]) # boolean predicate measuring greater than
eucl_dist = ltn.Function.Lambda(
      lambda inputs: tf.expand_dims(tf.norm(inputs[0]-inputs[1],axis=1),axis=1)
) # function measuring euclidian distance


Exists(d, 
      Forall([x,y],
            Eq([x,y]),
            mask = is_greater_than([d, eucl_dist([x,y])])
      )
)

ltn.Formula(tensor=0.7804231643676758, free_vars=[])

The guarded option is particularly useful to propagate gradients (see notebook on learning) over only a subset of the domains, that verifies the condition $m$.